## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-08-49-15 +0000,nyt,Can Indonesia Afford Prabowo’s Free School Lun...,https://www.nytimes.com/2025/07/03/world/asia/...
1,2025-07-10-08-48-50 +0000,nypost,FBI seeking military vet suspected in ICE ambu...,https://nypost.com/2025/07/10/us-news/fbi-seek...
2,2025-07-10-08-46-30 +0000,bbc,Search for survivors after Houthis sink second...,https://www.bbc.com/news/articles/c3071vp2d8yo
3,2025-07-10-08-22-55 +0000,bbc,'A huge explosion-like sound' - Survivors reca...,https://www.bbc.com/news/articles/cjel1g15qxpo
4,2025-07-10-08-22-24 +0000,nypost,Secret Service acquires iconic presidential Ma...,https://nypost.com/2025/07/10/us-news/secret-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
107,trump,15
124,ex,6
310,tariff,4
314,tariffs,4
68,deal,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
36,2025-07-10-00-26-49 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,31
28,2025-07-10-02-29-17 +0000,nypost,Secret Service suspends 6 agents assigned to p...,https://nypost.com/2025/07/09/us-news/secret-s...,30
50,2025-07-09-19-47-41 +0000,nyt,Can Canada Leverage Its Unique U.S. Relationsh...,https://www.nytimes.com/2025/07/09/world/canad...,30
34,2025-07-10-01-01-31 +0000,nyt,Trump Tariffs Aim to Settle Scores With Countr...,https://www.nytimes.com/2025/07/09/business/ec...,29
33,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo,29


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
36,31,2025-07-10-00-26-49 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
21,29,2025-07-10-04-39-28 +0000,nypost,Ex-CIA Director John Brennan says he’s had ‘no...,https://nypost.com/2025/07/10/us-news/ex-cia-d...
19,23,2025-07-10-04-42-44 +0000,nypost,Philadelphia workers and city reach deal to en...,https://nypost.com/2025/07/10/us-news/philadel...
17,18,2025-07-10-05-03-24 +0000,nypost,Jealous ex-wife allegedly spied on former husb...,https://nypost.com/2025/07/10/us-news/californ...
4,17,2025-07-10-08-22-24 +0000,nypost,Secret Service acquires iconic presidential Ma...,https://nypost.com/2025/07/10/us-news/secret-s...
24,16,2025-07-10-04-10-05 +0000,nypost,50 kids evacuated from Oklahoma Boys & Girls C...,https://nypost.com/2025/07/10/us-news/50-kids-...
29,16,2025-07-10-02-27-12 +0000,nypost,"Illinois father, 65, and his autistic son, 20,...",https://nypost.com/2025/07/09/us-news/illinois...
14,14,2025-07-10-05-23-26 +0000,nypost,Ex-California cop who allegedly partied at mus...,https://nypost.com/2025/07/10/us-news/californ...
3,13,2025-07-10-08-22-55 +0000,bbc,'A huge explosion-like sound' - Survivors reca...,https://www.bbc.com/news/articles/cjel1g15qxpo
6,13,2025-07-10-08-16-15 +0000,nypost,"Outlawz rapper Young Noble, who collaborated w...",https://nypost.com/2025/07/10/us-news/outlawz-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
